# Үй тапсырмасы: Банк клиенттерінің кетуін болжау (Customer Churn Prediction)

**Тақырыбы:** Нейрондық желілер негіздері (Classification).

**Мақсаты:**

1. Шынайы деректер жинағын (Dataset) өңдеу және талдау.
2. Категориялық деректерді цифрлық форматқа ауыстыру.
3. Терең оқыту (Deep Learning) моделін, яғни көпқабатты персептронды құру.
4. Модельдің дәлдігін бағалау.

**Деректер жинағы:**
Біз 10,000 банк клиенті туралы ақпаратты қолданамыз. Мақсат — клиенттің банк қызметінен бас тарту-тартпауын (`Exited` бағаны: 1 немесе 0) болжау.



## 1. Қысқаша теория

Бұл есеп — **бинарлы классификация** (екі класқа бөлу) мәселесі.

**Нейрондық желі архитектурасы:**

1. **Кіріс қабаты (Input Layer):** Клиенттің белгілеріне (жасы, жалақысы, елі, т.б.) сәйкес келетін нейрондар саны.
2. **Жасырын қабаттар (Hidden Layers):** Деректердегі күрделі заңдылықтарды анықтайды. Белсендіру функциясы ретінде әдетте **ReLU** () қолданылады.
3. **Шығыс қабаты (Output Layer):** Бинарлы классификация үшін 1 нейрон қолданылады. Белсендіру функциясы — **Sigmoid** (), себебі ол бізге 0 мен 1 арасындағы ықтималдықты береді.

In [2]:
# Қажетті кітапханаларды импорттау
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Кездейсоқ сандар генераторын бекіту (нәтиже тұрақты болу үшін)
tf.random.set_seed(42)
np.random.seed(42)

print("Кітапханалар сәтті жүктелді!")

Кітапханалар сәтті жүктелді!




## 2. Деректерді жүктеу және шолу

Біз деректерді GitHub репозиторийінен тікелей жүктейміз.


In [4]:
# Деректерді оқу
url = "https://raw.githubusercontent.com/erkansirin/Deep-Learning-Bootcamp/master/datasets/Churn_Modelling.csv"
df = pd.read_csv(url)

# Алғашқы 5 жолды шығару
df.head()

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Деректердің жалпы сипаттамасы
df.info()


## 3. Деректерді алдын ала өңдеу (Preprocessing)

Бұл кезеңде біз модельге қажетсіз бағандарды өшіріп, мәтіндік деректерді сандарға айналдырамыз.

**Тапсырма 1:** Төмендегі бағандар модельге пайдасыз, себебі олар клиенттің кетуіне әсер етпейді:

* `RowNumber`
* `CustomerId`
* `Surname`

Осы бағандарды өшіріңіз.


In [ ]:
# TODO: "RowNumber", "CustomerId", "Surname" бағандарын өшіріңіз (drop функциясын қолданыңыз)
df = df.drop(columns=[...]) # Осы жерді толтырыңыз

# Тексеру
print(df.shape)
df.head(3)



### Категориялық айнымалыларды кодтау

Бізде екі мәтіндік баған бар: `Geography` (Ел) және `Gender` (Жыныс). Нейрондық желі тек сандармен жұмыс істейді.

* `Gender` -> Label Encoding (Erkak=1, Ayel=0).
* `Geography` -> One-Hot Encoding (France, Germany, Spain бағандарына бөлу).

**Тапсырма 2:** `Gender` бағанын сандық форматқа, ал `Geography` бағанын One-Hot форматына ауыстырыңыз.


In [ ]:
# Gender бағанын кодтау (Label Encoding)
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

# TODO: Geography бағаны үшін One-Hot Encoding жасаңыз (pd.get_dummies қолданыңыз)
# drop_first=True параметрін қолдану ұсынылады (мультиколлинеарлықты болдырмау үшін)
df = pd.get_dummies(df, columns=[...], drop_first=True) # Осы жерді толтырыңыз

df.head()


## 4. Деректерді бөлу және масштабтау (Scaling)

Нейрондық желілер үшін деректердің масштабталуы өте маңызды. Мысалы, `EstimatedSalary` (мыңдаған) және `Age` (ондаған) әртүрлі диапазонда. Біз оларды бірдей диапазонға келтіруіміз керек.


In [ ]:
# Белгілер (X) мен нысаналы айнымалыны (y) бөлу
X = df.drop('Exited', axis=1)
y = df['Exited']

# TODO: Деректерді оқу (train) және тест (test) жиындарына бөліңіз (20% тест үшін)
# random_state=42 параметрін қолданыңыз
X_train, X_test, y_train, y_test = train_test_split(..., ..., test_size=0.2, random_state=42)

# Масштабтау (StandardScaler)
scaler = StandardScaler()

# TODO: scaler-ді X_train деректеріне үйретіп (fit), X_train мен X_test-ті түрлендіріңіз (transform)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(...) # Осы жерді толтырыңыз

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)



## 5. Нейрондық желіні құру

Біз `Sequential` моделін қолданамыз.

**Тапсырма 3:** Төмендегі архитектураны құрыңыз:

1. **Кіріс қабаты + 1-ші жасырын қабат:** Нейрондар саны (мысалы, 16 немесе 32), белсендіру функциясы 'relu'. `input_dim` параметрін дұрыс көрсетіңіз.
2. **2-ші жасырын қабат:** Нейрондар саны (мысалы, 8 немесе 16), белсендіру функциясы 'relu'.
3. **Шығыс қабаты:** 1 нейрон, белсендіру функциясы 'sigmoid' (бинарлы классификация үшін).



In [ ]:
model = Sequential()

# TODO: Қабаттарды қосыңыз (add әдісі арқылы)

# 1-қабат
model.add(Dense(units=..., activation='...', input_dim=X_train.shape[1])) # Осы жерді толтырыңыз

# 2-қабат (Жасырын)
model.add(Dense(units=..., activation='...')) # Осы жерді толтырыңыз

# 3-қабат (Шығыс)
model.add(Dense(units=1, activation='sigmoid'))

# Модель құрылымын қарау
model.summary()



## 6. Модельді компиляциялау және оқыту

* **Optimizer:** `adam` (ең кең таралған оптимизатор).
* **Loss Function:** `binary_crossentropy` (бинарлы классификация үшін стандарт).
* **Metrics:** `accuracy`.


In [ ]:
# Компиляция
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# TODO: Модельді оқыту (fit)
# batch_size=32, epochs=50 (немесе 100) қолданыңыз
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=..., # Осы жерді толтырыңыз
                    validation_data=(X_test, y_test),
                    verbose=1)



## 7. Нәтижелерді бағалау

Оқыту барысындағы дәлдік (accuracy) және шығын (loss) графиктерін салайық.


In [ ]:
# Графиктерді салу
plt.figure(figsize=(12, 5))

# Loss графигі
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy графигі
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()



## 8. Болжам жасау және Метрикалар

Модель 0 мен 1 арасындағы ықтималдықты қайтарады. Біз оны 0.5 шегін (threshold) қолданып, нақты класқа (0 немесе 1) айналдыруымыз керек.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Болжам жасау
y_pred_prob = model.predict(X_test)

# TODO: Ықтималдықты 0 немесе 1 класына айналдырыңыз (егер > 0.5 болса 1, әйтпесе 0)
y_pred = (y_pred_prob > 0.5).astype(int)

# Нәтижелер
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


## Қорытынды сұрақтар

Төмендегі сұрақтарға жауап жазыңыз (мәтін түрінде):

1. Модельдің тест деректеріндегі дәлдігі (accuracy) қандай болды?
2. Precision және Recall мәндерінің айырмашылығы неде және бұл есеп үшін қайсысы маңыздырақ деп ойлайсыз? (Банк үшін клиенттің кететінін алдын ала білу маңызды).
3. Егер `epochs` санын тым көп арттырсақ (мысалы, 100-ге), не болуы мүмкін? (Overfitting туралы ойланыңыз).